Bayes(Gauß) Klassifikator ist ein Klassifikator, der aus dem Satz von Bayes hergeleitet ist, er ordnet jedes Objekt der Klasse zu, zu der es mit der größten Wahrscheinlichkeit gehört. Häufig wird der Bayes-Klassifikator auch zur Beurteilung anderer Klassifikatoren verwendet.

Wir benutzen die Datenpunkte der Ziffern aus dem Datensatz ”Digits“. Und implementieren sechs binäre Klassifikatoren, die jeweils zwei Klassen voneinander unterscheiden können mittels dem Gauß-Klassifikator.

(Quelle: https://web.stanford.edu/~hastie/ElemStatLearn/)

In [1]:
import pandas as pd
import numpy as np

def file_get_contents(filename):
    return np.genfromtxt(filename, delimiter=',')

Laden Training Daten und Test Daten.

In [3]:
t3, t5 = file_get_contents('train.3'), file_get_contents('train.5')
t7, t8 = file_get_contents('train.7'), file_get_contents('train.8')

In [5]:
print(t5)

[[-1.    -1.    -1.    ... -0.671 -0.828 -1.   ]
 [-1.    -1.    -1.    ... -1.    -1.    -1.   ]
 [-1.    -1.    -1.    ... -1.    -1.    -1.   ]
 ...
 [-1.    -1.    -1.    ... -1.    -1.    -1.   ]
 [-1.    -1.    -1.    ... -1.    -1.    -1.   ]
 [-1.    -1.    -1.    ... -1.    -1.    -1.   ]]


Die Idee für Klassifizieren: berechnen wir für jede Klasse den Durchschnittsvektor und die Kovarianzmatrix, berechnen dann (Durchschnittsvektor / Kovarianzmatrix), damit wir die Wahrscheinlichkeit von jede Klassen bekommen.
Vorhersagen: wählen die Klasse mit höchster Wahrscheinlichkeit als Vorhersage.

Dafür brauchen wir Kovarianzmatrix und Durchschnittsvektor (diameter).

In [6]:
def diameter(images):      
    result = np.zeros((len(images[0])), dtype=np.float)

    for pixel in range(0, len(images[0])):  # bekommen an jedem Index von allen Sample die durchschnittliche Werte.
        sum = 0

        for image in range(0, len(images)):
            sum += images[image][pixel]

        result[pixel] = (sum / len(images))

    return result


def mycovariance(matrix):    # berechne die Kovarianzmatrix von unserer Trainingklasse
    middlevec = diameter(matrix)
    length = len(matrix)
    matrix1 = np.transpose(np.subtract(matrix, middlevec))
    matrix2 = np.transpose(matrix1)
    matrix = np.dot(matrix1, matrix2)
    return np.divide(matrix, length-1)

Jetzt berechnen wir die Wahrscheinlichkeit von jeder Klasse mit hilfe von Normalverteilung Berechnung.

In [10]:
def get_it_invertible(matrix):  # erzeuge einen invertierbaren Matrix
    a = 0.1
    id = np.identity(len(matrix))
    deter = np.linalg.det(matrix)

    while (deter == 0):    # wenn die Determinante 0 ist, gibt's keinen Inverse Matrix
        matrix = np.add(matrix, np.dot(a, id))
        deter = np.linalg.det(matrix)
    return matrix


def get_gauss(matrix, test):  
    length = len(test)
    sigma = mycovariance(matrix)
    sigmaInverse = np.linalg.inv(get_it_invertible(sigma)) # muss garantieren, das einen Inverse Matrix existiert
    pi = np.pi
    determinante = np.linalg.det(get_it_invertible(sigma))
    a = np.sqrt(pi * length * determinante)
    coefficient = 1 / a

    meanVector = diameter(matrix)
    sub = np.subtract(test, meanVector)
    exponent1 = np.dot(np.transpose(sub), sigmaInverse)
    exponent2 = np.matrix(np.dot(exponent1, sub))
    exponent = np.exp(np.dot(-0.5, exponent2)) # schrittweise durchführen laut der Formel
    return np.dot(coefficient, exponent)

In [11]:
def prediction(matrix1, matrix2, test):
    gauss1 = get_gauss(matrix1[0], test)
    gauss2 = get_gauss(matrix2[0], test)

    if (np.greater(gauss1, gauss2)): 
        return matrix1[1]
    else: return matrix2[1]

Schauen wir uns jetzt mal die Error Rate an.

In [13]:
def prediction_rate(matrix1, matrix2, test):
    length = len(test[0])
    error = 0
    for i in range (0, length):
        result = prediction(matrix1, matrix2, test[0][i])
        if (result != test[1]):
            error = error + 1    
    print('Error rate zwischen', matrix1[1], 'und', matrix2[1], 'ist', (error/length)*100 ,'%')
    return 'done'

print(prediction_rate([t3, 3], [t5, 5], [t3, 3]))
print(prediction_rate([t3, 3], [t7, 7], [t7, 7]))

Error rate zwischen 3 und 5 ist 0.303951367781155 %
done
Error rate zwischen 3 und 7 ist 0.15503875968992248 %
done
